# Stacks, Queues, Deques and Linked Lists

This notebook describes Stacks, Queues and Deques in Python. It covers the following:

1. Stack
2. Queue
3. Deque
4. Singly Linked List
5. Doubly Linked List

For each concept, there are Python examples which help illustrate the ideas.


## 1. Stack

![](https://github.com/MattScicluna/coding_tests_repo/blob/second_branch/5-Stacks_Queues_and_Deques/images/stacks.png)

Here is an implementation of a stack, where we adapt it from Python's `list` class 

### Adapter Pattern

![](./images/adapter.png)

In [1]:
class Empty(Exception):
    """Error attempting to access an element from an empty container."""
    pass

class Stack:
    def __init__(self):
        self._data = []

    def push(self, value):
        self._data.append(value)

    def pop(self):
        if not self.is_empty():
            return self._data.pop()
        return Empty

    def __len__(self):
        return len(self._data)

    def top(self):
        if not self.is_empty():
            return self._data[-1]
        return Empty

    def is_empty(self):
        return len(self._data)==0
    
    def __repr__(self):
        return str(self._data)

We instantiate a Stack and ensure that it works as expected

In [2]:
S = Stack()
out = S.push(5)
print('{} : {}'.format(out, S))
out = S.push(3)
print('{} : {}'.format(out, S))
out = len(S)
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.is_empty()
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.is_empty()
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.push(7)
print('{} : {}'.format(out, S))
out = S.push(9)
print('{} : {}'.format(out, S))
out = S.top()
print('{} : {}'.format(out, S))
out = S.push(4)
print('{} : {}'.format(out, S))
out = len(S)
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.push(6)
print('{} : {}'.format(out, S))
out = S.push(8)
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))

None : [5]
None : [5, 3]
2 : [5, 3]
3 : [5]
False : [5]
5 : []
True : []
<class '__main__.Empty'> : []
None : [7]
None : [7, 9]
9 : [7, 9]
None : [7, 9, 4]
3 : [7, 9, 4]
4 : [7, 9]
None : [7, 9, 6]
None : [7, 9, 6, 8]
8 : [7, 9, 6]


We can use our Stack to create a function which determines whether delimiters are properly matched

In [3]:
def is_matched(expr):
    """Return True if all delimiters are properly match; False otherwise."""
    lefty = '({['                                     # opening delimiters
    righty = ')}]'                                    # respective closing delims
    S = Stack()
    for e in expr:
        if e in lefty:
            S.push(e)
        elif e in righty:
            if S.is_empty():
                return False
            old_e = S.pop()
            if lefty.index(old_e) == righty.index(e): #  check correspondence
                pass                                  #  corresponding delimeters
            else:
                return False                          #  delimiters do not correspond
    return S.is_empty()

In [4]:
print(is_matched('(this is a {test}) []'))
print(is_matched('this is a test'))
print(is_matched('this is a test [(])'))
print(is_matched('[]this is a test {'))

True
True
False
False


This code runs in $O(n)$ time, since there are at most $n$ push and pop operations, and each have (amortized) complexity of $O(1)$

## 2. Queue

![](./images/queue.png)

We again implement a Queue by adapting it from Python's `list` class

In [5]:
import copy

class Queue:
    """
    FIFO queue implementation using a Python list as underlying storage.
    """
    DEFAULT_CAPACITY = 10
    def __init__(self):
        self._data = [None]*self.DEFAULT_CAPACITY         # reference to a list instance with a fixed capacity.
        self._size = 0                                    # integer representing the current number of elements stored in the queue (as opposed to the length of the data list).
        self._front = 0                                   # integer representing the index within data of the first element of the queue (assuming the queue is not empty).
    
    def _add_to_front(self, amount):
        if self.is_empty():
            return self._front + amount
        return (self._front + amount) % len(self._data)
    
    def enqueue(self, value):
        """Add an element to the back of queue"""
        #  increase size of array when out of space
        if len(self) == len(self._data):
            self._change_capacity(len(self._data)*2)
        self._data[self._add_to_front(self._size)] = value
        self._size += 1

    def dequeue(self):
        """
        Remove and return the first element of the queue (i.e., FIFO).
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            return Empty
        val = copy.deepcopy(self._data[self._front])
        self._data[self._front] = None                    # remove reference of object for Python garbage collection
        self._front = self._add_to_front(1)
        self._size -= 1
        #  decrease size of array when only using < 1/4
        if self._size < (len(self._data) // 4):
            self._change_capacity(len(self._data) // 2)
        return val

    def __len__(self):
        """Return the number of elements in the queue"""
        return self._size

    def first(self):
        """
        Return (but do not remove) the element at the front of the queue.
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            return Empty
        return self._data[self._front]

    def is_empty(self):
        """Return True if the queue is empty"""
        return len(self) == 0
    
    def __repr__(self):
        if self.is_empty():
            return '[]'
        string = '['
        for j in range(self._size):
            string += str(self._data[self._add_to_front(j)]) + ','
        string = string[:-1]
        string += ']'
        return string
    
    def _change_capacity(self, amount):
        """
        Change capacity of list by amount and shift indices to front
        Recall Dynamic Arrays from Chapter 5
        """
        new_data = [None]*amount
        i = 0
        for j in range(self._size):
            new_data[i] = self._data[self._add_to_front(j)]
            i += 1
        self._data = new_data
        self._front = 0

We instantiate a Queue to make sure it works as expected

In [6]:
Q = Queue()
out = Q.enqueue(5)
print('{} : {}'.format(out, Q))
out = Q.enqueue(3)
print('{} : {}'.format(out, Q))
out = len(Q)
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))
out = Q.is_empty()
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))
out = Q.is_empty()
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))
out = Q.enqueue(7)
print('{} : {}'.format(out, Q))
out = Q.enqueue(9)
print('{} : {}'.format(out, Q))
out = Q.first()
print('{} : {}'.format(out, Q))
out = Q.enqueue(4)
print('{} : {}'.format(out, Q))
out = len(Q)
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))

None : [5]
None : [5,3]
2 : [5,3]
5 : [3]
False : [3]
3 : []
True : []
<class '__main__.Empty'> : []
None : [7]
None : [7,9]
7 : [7,9]
None : [7,9,4]
3 : [7,9,4]
7 : [9,4]


We additionally check that it can handle dynamic resizing (since it is using a list with fixed capacity which it increases/decreases as needed)

We can see that the internal representation of the queue is faithful as is the string representation.

In [7]:
Q = Queue()

for i in range(5):
    Q.enqueue(i)
for _ in range(5):
    Q.dequeue()
for i in range(5, 15):
    Q.enqueue(i)
for _ in range(8):
    Q.dequeue()

print(Q._data)
print(Q)

[None, 13, 14, None, None, None, None, None]
[13,14]


## 3. Deque

These are double ended queues, so you can add and delete from both sides.

They are already implemented in Pythons `collections` module

In [8]:
from collections import deque

In [9]:
D = deque()
out = D.appendleft(5)           # add to beginning
print('{} : {}'.format(out, D))
out = D.append(3)               # add to end
print('{} : {}'.format(out, D))
out = len(D)                    # number of elements
print('{} : {}'.format(out, D))
out = D.append(5)
print('{} : {}'.format(out, D))
out = D.count(5)                # count number of matches for e=5
print('{} : {}'.format(out, D))
out = D.popleft()               # remove from beginning
print('{} : {}'.format(out, D))
out = D.pop()                   # remove from end
print('{} : {}'.format(out, D))
out = D.append(D[0])
print('{} : {}'.format(out, D))
out = D.appendleft(D[-1])
print('{} : {}'.format(out, D))
D[1] = 999
print('{} : {}'.format(out, D))
out = D.rotate(2)               # circularly shift rightward k=2 steps
print('{} : {}'.format(out, D))
out = D.remove(999)             # remove first matching element
print('{} : {}'.format(out, D))
out = D.clear()                 # clear all contents
print('{} : {}'.format(out, D))

None : deque([5])
None : deque([5, 3])
2 : deque([5, 3])
None : deque([5, 3, 5])
2 : deque([5, 3, 5])
5 : deque([3, 5])
5 : deque([3])
None : deque([3, 3])
None : deque([3, 3, 3])
None : deque([3, 999, 3])
None : deque([999, 3, 3])
None : deque([3, 3])
None : deque([])


## 4. Singly Linked Lists

Note: linked lists are an alternative to array based sequences (which we've used in our above implementations of Stacks and Queues)

![](./images/SLL.png)

We implement a Stack using a Linked List

In [10]:
class LinkedStack:
    """
    LIFO Stack implementation using a singly linked list for storage.
    Notice how we orient the linkedlist s.t. the head of the list 
    corresponds to the top of the stack
    """
    #-------------------------- nested Node class --------------------------
    class _Node:
        """Lightweight, nonpublic class for storing a singly linked node."""
        __slots__ = '_element' , '_next'     # streamline memory usage
        def __init__(self, element, next):   # initialize node’s fields
            self._element = element          # reference to user’s element
            self._next = next                # reference to next node
        def __repr__(self):
            return str(self._element)
    
    def __init__(self):
        """
        Create an empty stack
        """
        self._head = None                    # reference to the node at the head of the list (or None, if the stack is empty)
        self._size = 0                       # current number of elements

    def push(self, e):
        """
        Add element e to the top of the stack
        """
        self._head = self._Node(e, self._head) # create and link a new node
        self._size += 1

    def pop(self):
        """
        Remove and return the element from the top of the stack (i.e., LIFO).
        Raise Empty exception if the stack is empty.
        """
        if not self.is_empty():
            to_return = copy.deepcopy(self._head)
            self._head = self._head._next
            self._size -= 1
            return to_return._element
        return Empty

    def __len__(self):
        """
        Return the number of elements in the stack
        """
        return self._size

    def top(self):
        """
        Return (but do not remove) the element at the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if not self.is_empty():
            return self._head._element
        return Empty

    def is_empty(self):
        """
        Return True if the stack is empty
        """
        return self._head is None
    
    def __repr__(self):
        if self.is_empty():
            return '[]'
        string = '['
        curr_node = self._head
        for _ in range(self._size):
            string += str(curr_node) + ','
            curr_node = curr_node._next
        string = string[:-1]
        string += ']'
        return string

We run the same tests to make sure our `LinkedStack` works the same way as the array-based `Stack`

In [11]:
S = LinkedStack()
out = S.push(5)
print('{} : {}'.format(out, S))
out = S.push(3)
print('{} : {}'.format(out, S))
out = len(S)
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.is_empty()
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.is_empty()
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.push(7)
print('{} : {}'.format(out, S))
out = S.push(9)
print('{} : {}'.format(out, S))
out = S.top()
print('{} : {}'.format(out, S))
out = S.push(4)
print('{} : {}'.format(out, S))
out = len(S)
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))
out = S.push(6)
print('{} : {}'.format(out, S))
out = S.push(8)
print('{} : {}'.format(out, S))
out = S.pop()
print('{} : {}'.format(out, S))

None : [5]
None : [3,5]
2 : [3,5]
3 : [5]
False : [5]
5 : []
True : []
<class '__main__.Empty'> : []
None : [7]
None : [9,7]
9 : [9,7]
None : [4,9,7]
3 : [4,9,7]
4 : [9,7]
None : [6,9,7]
None : [8,6,9,7]
8 : [6,9,7]


We implement the Queue using a Linked List

In [12]:
class LinkedQueue:
    """
    FIFO queue implementation using a singly linked list for storage.
    Notice how we orient the linkedlist s.t. the head of the list 
    corresponds to the front of the queue (otherwise, dequeueing would be O(n))
    """
    #-------------------------- nested Node class --------------------------
    class _Node:
        """Lightweight, nonpublic class for storing a singly linked node."""
        __slots__ = '_element' , '_next'     # streamline memory usage
        def __init__(self, element, next):   # initialize node’s fields
            self._element = element          # reference to user’s element
            self._next = next                # reference to next node
        def __repr__(self):
            return str(self._element)

    def __init__(self):
        """Create an empty queue."""
        self._head = None
        self._tail = None
        self._size = 0                       # number of queue elements

    def enqueue(self, value):
        """Add an element to the back of queue"""
        new = self._Node(value, None)        # create node pointing to None
        if self.is_empty():
            self._head = new
            self._tail = new
        else:
            self._tail._next = new               # make current tail point to new node
            self._tail = new                     # set tail node to the new node
        self._size += 1

    def dequeue(self):
        """
        Remove and return the first element of the queue (i.e., FIFO).
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            return Empty
        val = copy.deepcopy(self._head._element)
        self._head = self._head._next
        self._size -= 1
        return val

    def __len__(self):
        """Return the number of elements in the queue"""
        return self._size

    def first(self):
        """
        Return (but do not remove) the element at the front of the queue.
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            return Empty
        return self._head._element

    def is_empty(self):
        """Return True if the queue is empty"""
        return len(self) == 0
    
    def __repr__(self):
        if self.is_empty():
            return '[]'
        string = '['
        curr_node = self._head
        for _ in range(self._size):
            string += str(curr_node) + ','
            curr_node = curr_node._next
        string = string[:-1]
        string += ']'
        return string

Again, we ensure that the output for the `LinkedQueue` matches with the array-based `Queue`

In [13]:
Q = LinkedQueue()
out = Q.enqueue(5)
print('{} : {}'.format(out, Q))
out = Q.enqueue(3)
print('{} : {}'.format(out, Q))
out = len(Q)
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))
out = Q.is_empty()
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))
out = Q.is_empty()
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))
out = Q.enqueue(7)
print('{} : {}'.format(out, Q))
out = Q.enqueue(9)
print('{} : {}'.format(out, Q))
out = Q.first()
print('{} : {}'.format(out, Q))
out = Q.enqueue(4)
print('{} : {}'.format(out, Q))
out = len(Q)
print('{} : {}'.format(out, Q))
out = Q.dequeue()
print('{} : {}'.format(out, Q))

None : [5]
None : [5,3]
2 : [5,3]
5 : [3]
False : [3]
3 : []
True : []
<class '__main__.Empty'> : []
None : [7]
None : [7,9]
7 : [7,9]
None : [7,9,4]
3 : [7,9,4]
7 : [9,4]


In [14]:
Q = LinkedQueue()

for i in range(5):
    Q.enqueue(i)
for _ in range(5):
    Q.dequeue()
for i in range(5, 15):
    Q.enqueue(i)
for _ in range(8):
    Q.dequeue()

print(Q)

[13,14]


### Circularly Linked List

![](./images/CLL.png)

## 5. Doubly Linked List

![](./images/DLL.png)

We create a class `_DoublyLinkedBase` which our class `LinkedDeque` inherits from. 

`LinkedDeque` is our implementation of a Deque using a Doubly Linked List.

In [15]:
class _DoublyLinkedBase:
    """A base class providing a doubly linked list representation"""

    class _Node:
        """Lightweight, nonpublic class for storing a doubly linked node."""
        slots = '_element' , '_prev' , '_next' # streamline memory
        def __init__(self, element, prev, next): # initialize node’s fields
            self._element = element # user’s element
            self._prev = prev # previous node reference
            self._next = next # next node reference
        def __repr__(self):
            return str(self._element)

    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, self._header, None)
        self._header._next = self._trailer
        self._size = 0

    def __len__(self):
        """Return the number of elements in the queue"""
        return self._size

    def is_empty(self):
        """Return True if the queue is empty"""
        return len(self) == 0

    def _insert_between(self, e, predecessor, successor):
        """Add element e between two existing nodes and return new node."""
        new = self._Node(e, predecessor, successor)
        predecessor._next = new
        successor._prev = new
        self._size += 1
        return new

    def _delete_node(self, node):
        """Delete nonsentinel node from the list and return its element."""
        previous = node._prev
        successor = node._next
        previous._next = successor
        successor._prev = previous
        self._size -= 1
        element = node._element
        node._prev = node._next = node._element = None  # indicates depricate node, and removes any references to help Python with garbage collection
        return element


In [16]:
class LinkedDeque(_DoublyLinkedBase): # note the use of inheritance
    """Double-ended queue implementation based on a doubly linked list."""
    
    def first(self):
        """Return (but do not remove) the element at the front of the deque."""
        if self.is_empty():
            return Empty
        return self._header._next._element
        

    def last(self):
        """Return (but do not remove) the element at the back of the deque."""
        if self.is_empty():
            return Empty
        return self._trailer._prev._element

    def insert_first(self, e):
        """Add an element to the front of the deque."""
        self._insert_between(e, self._header, self._header._next)

    def insert_last(self, e):
        """Add an element to the back of the deque."""
        self._insert_between(e, self._trailer._prev, self._trailer)
    
    def delete_first(self):
        """
        Remove and return the element from the front of the deque.
        Raise Empty exception if the deque is empty.
        """
        if self.is_empty():
            return Empty
        return self._delete_node(self._header._next)
    
    def delete_last(self):
        """
        Remove and return the element from the front of the deque.
        Raise Empty exception if the deque is empty.
        """
        if self.is_empty():
            return Empty
        return self._delete_node(self._trailer._prev)
    
    def __repr__(self):
        if self.is_empty():
            return '[]'
        else:
            string = '['
            curr_node = self._header._next
            for _ in range(self._size):
                string += str(curr_node) + ','
                curr_node = curr_node._next
            string = string[:-1]
            return string + ']'

We instantiate our `LinkedDeque` object and ensure that it is working properly.

In [17]:
L = LinkedDeque()
print(L.insert_first(1), L)
print(L.insert_last(2), L)
print(L.first(), L)
print(L.insert_last(3), L)
print(L.delete_last(), L)
print(L.delete_first(), L)
print(L.delete_first(), L)
print(L.delete_first(), L)

None [1]
None [1,2]
1 [1,2]
None [1,2,3]
3 [1,2]
1 [2]
2 []
<class '__main__.Empty'> []


Notice how a new element appears in the list! (when we `_insert_between` method improperly!)

This is why we shouldn't try to call access hidden attributes/methods

In [18]:
L.insert_first(66)
L._insert_between(4, L._header, L._Node(77, None, None)) #  calling method with nodes not in linkedlist!

4

Comparison of Array-Based and Linked structures:

![](./images/table.png)

Note: 
* Operations with equivalent asymptotic bounds typically run a constant factor more efficiently with an array-based structure versus a linked structure(since more CPU operations like instantiating new nodes, linking, etc.…)
* BUT, linked structures have better worst-case bounds, since no resizing!
